# Hybrid Search LangChain

In [ ]:
%pip install --upgrade --quiet pinecone-text pinecone-notebooks pinecone

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
api_key = os.getenv("PINECONE_API")

In [2]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
from pinecone import Pinecone, ServerlessSpec
index_name = "hybrid-search-langchain-pinecone"
# initialize the Pinecone client
pc = Pinecone(api_key=api_key)
# create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, # dimension of dense vector
        metric='dotproduct', # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud='aws',region='us-east-1')
    )


In [4]:
index = pc.Index(index_name)
print(index)

In [ ]:
## vector embedding and sparse matric 
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [6]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

In [7]:
sentences = [
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orleans"
]
# tfidf values on these sentences
bm25_encoder.fit(sentences)
# Store the values to a json file
bm25_encoder.dump("bm25_values.json")
# load to BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 83.33it/s]


In [8]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [10]:
print(retriever)

embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False) sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x0000018DEF729E10> index=<pinecone.db_data.index.Index object at 0x0000018DFD70EF10>


In [11]:
retriever.add_texts([
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orleans"
])

100%|██████████| 1/1 [00:00<00:00,  1.00it/s]


In [14]:
retriever.invoke("What city did I visit in 2021")

[Document(metadata={'score': 0.459573388}, page_content='In 2021, I visited New Orleans'),
 Document(metadata={'score': 0.326009393}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.29625532}, page_content='In 2023, I visited Paris')]